In [1]:
import os, sys, json, argparse
import pandas as p
import numpy as np
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from string import punctuation

import urllib.request, html5lib
from bs4 import BeautifulSoup, Tag, UnicodeDammit

In [2]:
#first i found a routine that extracted a json version of a script and converted it something usable
#used the same author to extract the scripts from http://www.imsdb.com/scripts into a json file
#the parsing routine started out as an interactive py and i converted it to a batch process. 

SW5 = '/Users/kenmckee/Desktop/SWE5'
SW7 = '/Users/kenmckee/Desktop/SWE7ANA'

with open('/Users/kenmckee/Desktop/SWE5') as fd:
    script = json.load(fd)
    
    print(script)

{'movie_url': 'https://www.imsdb.com/scripts/Star-Wars-A-New-Hope.html', 'movie_title': 'STAR WARS- EPISODE V - A NEW HOPE', 'movie_year': '1977', 'movie_script': [{'type': 'introduction', 'text': "STAR WARS Episode IV A NEW HOPE From the JOURNAL OF THE WHILLS by George Lucas Revised Fourth Draft January 15, 1976 LUCASFILM LTD. A long time ago, in a galaxy far, far, away... A vast sea of stars serves as the backdrop for the main title. War drums echo through the heavens as a rollup slowly crawls into infinity. It is a period of civil war. Rebel spaceships, striking from a hidden base, have won their first victory against the evil Galactic Empire. During the battle, Rebel spies managed to steal secret plans to the Empire's ultimate weapon, the Death Star, an armored space station with enough power to destroy an entire planet. Pursued by the Empire's sinister agents, Princess Leia races home aboard her starship, custodian of the stolen plans that can save her people and restore freedom t

In [3]:
BLOCK_TYPES=['character', 'speech', 'stage direction', 'location']
CHARACTER=0
SPEECH=1
DIRECTIONS=2
LOCATION=3

In [4]:
#adapted this routine from https://github.com/Adrien-Luxey/Da-Fonky-Movie-Script-Parser

def extract_characters(script):
    '''
    Extracts the (unique) characters list from the script
    '''
    characters=[]
    for block in script['movie_script']:
        if(block['type'] == BLOCK_TYPES[SPEECH]):
            character = block['character']
            if not character in characters:
                characters.append(character)

    return characters

def extract_locations(script):
    '''
    Extracts the (unique) locations list from the script
    '''
    locations=[]
    for block in script['movie_script']:
        if(block['type'] == BLOCK_TYPES[LOCATION]):
            location = block['text']
            if not location in locations:
                locations.append(location)

    return locations

def extract_directions(script):
    '''
    Extracts the stage directions list from the script
    '''
    directions=[]
    for block in script['movie_script']:
        if(block['type'] == BLOCK_TYPES[DIRECTIONS]):
            directions.append(block['text'])

    return directions

def extract_speech_given_character(script, character, strict_match=False):
    '''
    Extracts the given character's utterances from the script

    If strict_match is True, we will only extract utterances that perfectly match (==) the parameter;
    otherwise, we will extract utterances whose character partly matches (in) the parameter.
    In both cases, the match is case-insensitive.

    Also asks the user wether one wants to keep the character's name before each utterance.
    '''

    speeches=[]
    for block in script['movie_script']:
        if( block['type'] == BLOCK_TYPES[SPEECH] and
            (strict_match and (character.lower() == block['character'].lower()) or
             not strict_match and (character.lower() in block['character'].lower())) ):
            if( keep_character_name ):
                speeches.append(block['character'])
            speeches.append(block['text'])

    return speeches

def extract_all_characters_speech(script):
    '''
    Extracts all speeches from the script
    '''
    return extract_speech_given_character(script, '')

def extract_speech_asking_user(script):
    '''
    Extracts utterances by asking the user which character one wants to get

    Also asks wether the user wants a perfect (==) or partial (in) match.
    '''
    character = input_string('Please provide the name of the character: ')

    strict_match = False
    answer = input('Do you want utterances of this exact character (or any character that matches \'{}\')? (y/N) '.format(character))
    if( answer == 'y' or answer =='Y' ):
        strict_match = True

    return extract_speech_given_character(script, character, strict_match)

def extract_speech_using_characters_list(script):
    '''
    Extracts utterances by providing the user with the characters list

    Also asks wether the user wants a perfect (==) or partial (in) match.
    '''
    characters = sorted(extract_characters(script))
    character = characters[input_from_list('Please choose a character:', characters)]

    strict_match = False
    answer = input('Do you want utterances of this exact character (or any character that matches \'{}\')? (y/N) '.format(character))
    if( answer == 'y' or answer =='Y' ):
        strict_match = True

    return extract_speech_given_character(script, character, strict_match)

def extract_speech(script):
    '''
    Asks the user which speeches one wants to extract, and calls the appropriate function
    '''
    speech=[]

    choices = ['all characters',
               'give the character\'s name',
               'choose from the characters list']

    action = input_from_list("Which character speeches do you want to extract?", choices)

    if( action == 0 ):
        return extract_all_characters_speech(script)
    elif( action == 1 ):
        return extract_speech_asking_user(script)
    elif( action == 2 ):
        return extract_speech_using_characters_list(script)





In [5]:
keep_character_name = True
strict_match = False

result_character = extract_characters(script)
result_direction = extract_directions(script)
result_location = extract_locations(script)
result_speech = extract_all_characters_speech(script)
print(result_speech)

#print(result_location)
#print(result_location)




['THREEPIO', "Did you hear that? They've shut down the main reactor. We'll be destroyed for sure. This is madness!", 'THREEPIO', "We're doomed!", 'THREEPIO', "There'll be no escape for the Princess this time.", 'THREEPIO', "What's that?", 'THREEPIO', 'I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...', 'LUKE', 'Hurry up! Come with me! What are you waiting for?! Get in gear!', 'THREEPIO', 'Artoo! Artoo-Detoo, where are you?', 'THREEPIO', 'At last! Where have you been?', 'THREEPIO', "They're heading in this direction. What are we going to do? We'll be sent to the spice mine of Kessel or smashed into who knows what!", 'THREEPIO', 'Wait a minute, where are you going?', 'IMPERIAL OFFICER', 'The Death Star plans are not in the main computer.', 'VADER', 'Where are those transmissions you intercepted?', 'VADER', 'What have you done with those plans?', 'REBEL OFFICER', 'We intercepted no transmissions. Aaah... This is a consular ship. Were 

In [6]:
keep_character_name = True
strict_match = False

text=[]
characters=[]
for character in result_character:
    for block in script['movie_script']:
        if( block['type'] == BLOCK_TYPES[SPEECH] and
            (strict_match and (character.lower() == block['character'].lower()) or
             not strict_match and (character.lower() in block['character'].lower())) ):
            if( keep_character_name ):
                characters.append(block['character'])
                text.append(block['text'])

In [7]:
textl = []
for line in text:
        line2 = line.replace('\n',' ') 
        textl.append(line2)
        
print(textl)

["Did you hear that? They've shut down the main reactor. We'll be destroyed for sure. This is madness!", "We're doomed!", "There'll be no escape for the Princess this time.", "What's that?", 'I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...', 'Artoo! Artoo-Detoo, where are you?', 'At last! Where have you been?', "They're heading in this direction. What are we going to do? We'll be sent to the spice mine of Kessel or smashed into who knows what!", 'Wait a minute, where are you going?', "Hey, you're not permitted in there. It's restricted. You'll be deactivated for sure..", "Don't call me a mindless philosopher, you overweight glob of grease! Now come out before somebody sees you.", "Secret mission? What plans? What are you talking about? I'm not getting in there!", "I'm going to regret this.", "That's funny, the damage doesn't look as bad from out here.", 'Are you sure this things safe?', "How did I get into this mess? I really don

In [8]:
type(textl)
textll = " ".join(textl)
type(textll)

str

In [9]:
#print(allwords_tokenized)

print(textll)

Did you hear that? They've shut down the main reactor. We'll be destroyed for sure. This is madness! We're doomed! There'll be no escape for the Princess this time. What's that? I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister... Artoo! Artoo-Detoo, where are you? At last! Where have you been? They're heading in this direction. What are we going to do? We'll be sent to the spice mine of Kessel or smashed into who knows what! Wait a minute, where are you going? Hey, you're not permitted in there. It's restricted. You'll be deactivated for sure.. Don't call me a mindless philosopher, you overweight glob of grease! Now come out before somebody sees you. Secret mission? What plans? What are you talking about? I'm not getting in there! I'm going to regret this. That's funny, the damage doesn't look as bad from out here. Are you sure this things safe? How did I get into this mess? I really don't know how. We seem to be made to suffer. It's

In [10]:
def tokenize():
    if textll is not None:
        words = textll.lower().split()
        return words
    else:
        return None

def map_book(tokens):
    hash_map = {}

    if tokens is not None:
        for element in tokens:
            # Remove Punctuation
            word = element.replace(",","")
            word = word.replace(".","")

            # Word Exist?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map
    else:
        return None

In [11]:
import operator
# Tokenize the Book
textlll = tokenize()


# Create a Hash Map (Dictionary)
map = map_book(textlll)

smap = sorted(map.items(), key=operator.itemgetter(1), reverse=True)

print(smap)

[('the', 433), ('you', 363), ('to', 309), ('i', 289), ('a', 234), ('of', 162), ('in', 140), ('this', 130), ('your', 124), ('be', 118), ('it', 118), ('is', 113), ('and', 95), ('on', 94), ('what', 87), ('have', 85), ('for', 84), ('he', 84), ('that', 80), ('are', 79), ("don't", 74), ("i'm", 74), ('all', 74), ('not', 70), ("it's", 70), ('my', 68), ('with', 68), ('we', 66), ('going', 65), ('me', 65), ('(over', 65), ('here', 61), ('get', 61), ('can', 59), ('but', 57), ('if', 56), ('luke', 55), ('will', 52), ('there', 51), ('do', 51), ('sir', 47), ('no', 46), ('come', 46), ('out', 46), ('right', 45), ("you're", 44), ('was', 44), ('see', 43), ('now', 41), ('know', 41), ('got', 41), ('just', 40), ('about', 40), ('one', 40), ('him', 40), ('up', 38), ('think', 37), ("we're", 36), ('well', 36), ('us', 35), ('where', 34), ('take', 34), ('them', 34), ('they', 34), ('at', 33), ('go', 33), ('like', 32), ('so', 32), ('speaker)', 32), ('has', 31), ('only', 30), ("can't", 30), ('red', 30), ("i've", 29), 

In [12]:
#token count
len(textlll)
len(textlll)


12158

In [13]:
word_list = ['the','you','too']
# Show Word Information
for word in word_list:
    print('Word: [' + word + '] Frequency: ' + str(map[word]))

#print(tokens)

Word: [the] Frequency: 433
Word: [you] Frequency: 363
Word: [too] Frequency: 23


In [14]:
#Term Frequency Analysis

stoplistx = []
import nltk
from nltk.probability import FreqDist
ndist = FreqDist(textlll)
nitems = ndist.most_common(70)
for item in nitems:
    print (item[0], '\t', item[1])
    stoplistx.append(item[0])
print(stoplistx)
type(stoplistx)

the 	 433
you 	 311
to 	 306
i 	 286
a 	 234
of 	 161
in 	 128
your 	 124
this 	 117
be 	 116
is 	 107
and 	 94
what 	 87
it 	 85
have 	 84
he 	 84
for 	 81
on 	 77
are 	 76
don't 	 74
i'm 	 74
that 	 74
all 	 72
it's 	 70
not 	 68
with 	 68
we 	 66
my 	 66
(over 	 65
going 	 63
get 	 61
can 	 57
if 	 56
but 	 55
will 	 51
do 	 48
come 	 46
you're 	 44
was 	 42
out 	 40
got 	 40
just 	 39
see 	 39
me 	 37
we're 	 36
about 	 36
no 	 35
where 	 34
you. 	 34
think 	 34
there 	 34
take 	 34
they 	 34
at 	 32
like 	 32
speaker) 	 32
has 	 31
only 	 30
can't 	 30
know 	 29
well, 	 29
been 	 29
here 	 29
one 	 29
red 	 29
i've 	 28
more 	 28
luke, 	 28
as 	 27
our 	 27
['the', 'you', 'to', 'i', 'a', 'of', 'in', 'your', 'this', 'be', 'is', 'and', 'what', 'it', 'have', 'he', 'for', 'on', 'are', "don't", "i'm", 'that', 'all', "it's", 'not', 'with', 'we', 'my', '(over', 'going', 'get', 'can', 'if', 'but', 'will', 'do', 'come', "you're", 'was', 'out', 'got', 'just', 'see', 'me', "we're", 'about', 

list

In [15]:
#Conditional Probabilities


from collections import defaultdict

def build_conditional_probabilities(corpus):
	"""
	The function takes as its input a corpus string (words separated by 
	spaces) and returns a 2D dictionnary of probabilities P(next|current) of
	seeing a word "next" conditionnaly to seeing a word "current". 
	"""

	# First we parse the string to build a double dimension dictionnary that
	# returns the conditional probabilities.

	# We parse the string to build a first dictionnary indicating for each
	# word, what are the words that follow it in the string. Repeated next
	# words are kept so we use a list and not a set. 

	tokenized_string = corpus.split()
	previous_word = ""
	dictionnary = defaultdict(list)

	for current_word in tokenized_string:
		if previous_word != "":
			dictionnary[previous_word].append(current_word)
		previous_word = current_word
		
	# We know parse dictionnary to compute the probability each observed
	# next word for each word in the dictionnary. 

	for key in dictionnary.keys():
		next_words = dictionnary[key]
		unique_words = set(next_words) # removes duplicated
		nb_words = len(next_words)
		probabilities_given_key = {}
		for unique_word in unique_words:
			probabilities_given_key[unique_word] = \
				float(next_words.count(unique_word)) / nb_words
		dictionnary[key] = probabilities_given_key

	return dictionnary


def bigram_next_word_predictor(conditional_probabilities, current, next_candidate):
	"""
	The function takes as its input a 2D dictionnary of probabilities 
	P(next|current) of seeing a word "next" conditionnaly to seeing a word 
	"current", the current word being read, and a next candidate word, and
	returns P(next_candidate|current).
	"""

	# We look for the probability corresponding to the 
	# current -> next_candidate pair

	if current in conditional_probabilities:
		if next_candidate in conditional_probabilities[current]:
			return conditional_probabilities[current][next_candidate]

	#If current -> next_candidate pair has not been observed in the corpus,
	#the corresponding dictionnary keys will not be defined. We return 
	#a probability 0.0

	return 0.0

# An example corpus to try out the function
# We call the conditional probability dictionnary builder function
conditional_probabilities = build_conditional_probabilities(textll)
#type(conditional_probabilities)
#print(conditional_probabilities)

print(conditional_probabilities['Force'])
print(conditional_probabilities['Force!'])
print(conditional_probabilities['Force,'])
print(conditional_probabilities['Force.'])

{'and': 0.0625, 'can': 0.0625, 'will': 0.4375, 'flowing': 0.0625, 'is': 0.1875, 'if': 0.0625, 'be': 0.125}
[]
{'do': 0.2, 'Luke.': 0.8}
{'Obi-Wan': 0.25, 'Well,': 0.25, 'The': 0.25, 'I': 0.25}


In [16]:
bigramx = bigram_next_word_predictor(conditional_probabilities, "the", "Force")
bigramy = bigram_next_word_predictor(conditional_probabilities, "the", "force")



#print(conditional_probabilities)
print(bigramx)
print(bigramy)

0.031746031746031744
0.0


In [17]:
# this function takes a string and a list of words as parameters.
#   It will return all the words in the list that contain the string as a substring
def searchstring (substring, wordlist):
    # initialize the result
    result = [ ]
    #  loop over all the words
    for word in wordlist:
        # test each word if it contains the substring
        if substring in word:
            # add it to the result
            result.append(word)
    return result


In [18]:
result = searchstring('Force', textl)

for string in result:
    print(string)

    #print(result)

The Force?
I want to come with you to Alderaan. There's nothing here for me now. I want to learn the ways of the Force and become a Jedi like my father.
You don't believe in the Force, do you?
Don't be too proud of this technological terror you've constructed. The ability to destroy a planet is insignificant next to the power of the Force.
A tremor in the Force. The last time I felt it was in the presence of my old master.
Don't underestimate the power of the Force.
Obi-Wan is here. The Force is with him.
The Force is strong with this one!
A young Jedi named Darth Vader, who was a pupil of mine until he turned to evil, helped the Empire hunt down and destroy the Jedi Knights. He betrayed and murdered your father. Now the Jedi are all but extinct. Vader was seduced by the dark side of the Force.
Well, the Force is what gives a Jedi his power. It's an energy field created by all living things. It surrounds us and penetrates us. It binds the galaxy together.
You must learn the ways of the

In [19]:
# function that takes a word and returns true if it consists only
#   of non-alphabetic characters

def alpha_filter(w):
  # pattern to match a word of non-alphabetical characters
    pattern = re.compile('^[^a-z]+$')
    if (pattern.match(w)):
        return True
    else:
        return False


In [20]:
alphawords = [w for w in textlll if not alpha_filter(w)]
print(len(alphawords))
print(alphawords[:100])


12151
['did', 'you', 'hear', 'that?', "they've", 'shut', 'down', 'the', 'main', 'reactor.', "we'll", 'be', 'destroyed', 'for', 'sure.', 'this', 'is', 'madness!', "we're", 'doomed!', "there'll", 'be', 'no', 'escape', 'for', 'the', 'princess', 'this', 'time.', "what's", 'that?', 'i', 'should', 'have', 'known', 'better', 'than', 'to', 'trust', 'the', 'logic', 'of', 'a', 'half-sized', 'thermocapsulary', 'dehousing', 'assister...', 'artoo!', 'artoo-detoo,', 'where', 'are', 'you?', 'at', 'last!', 'where', 'have', 'you', 'been?', "they're", 'heading', 'in', 'this', 'direction.', 'what', 'are', 'we', 'going', 'to', 'do?', "we'll", 'be', 'sent', 'to', 'the', 'spice', 'mine', 'of', 'kessel', 'or', 'smashed', 'into', 'who', 'knows', 'what!', 'wait', 'a', 'minute,', 'where', 'are', 'you', 'going?', 'hey,', "you're", 'not', 'permitted', 'in', 'there.', "it's", 'restricted.', "you'll"]


In [21]:
nltkstopwords = nltk.corpus.stopwords.words('english')
print(len(nltkstopwords))
print(nltkstopwords)


179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [22]:
print(len(textlll))
print("")
print(textlll[:100])
print("")
print(textlll[12148:12158])


12158

['did', 'you', 'hear', 'that?', "they've", 'shut', 'down', 'the', 'main', 'reactor.', "we'll", 'be', 'destroyed', 'for', 'sure.', 'this', 'is', 'madness!', "we're", 'doomed!', "there'll", 'be', 'no', 'escape', 'for', 'the', 'princess', 'this', 'time.', "what's", 'that?', 'i', 'should', 'have', 'known', 'better', 'than', 'to', 'trust', 'the', 'logic', 'of', 'a', 'half-sized', 'thermocapsulary', 'dehousing', 'assister...', 'artoo!', 'artoo-detoo,', 'where', 'are', 'you?', 'at', 'last!', 'where', 'have', 'you', 'been?', "they're", 'heading', 'in', 'this', 'direction.', 'what', 'are', 'we', 'going', 'to', 'do?', "we'll", 'be', 'sent', 'to', 'the', 'spice', 'mine', 'of', 'kessel', 'or', 'smashed', 'into', 'who', 'knows', 'what!', 'wait', 'a', 'minute,', 'where', 'are', 'you', 'going?', 'hey,', "you're", 'not', 'permitted', 'in', 'there.', "it's", 'restricted.', "you'll"]

['look', 'out!', "we'll", 'get', 'to', 'work', 'on', 'him', 'right', 'away.']


In [23]:
morestopwords = stoplistx


In [24]:
stopwords =  morestopwords
print(len(stopwords))
print(stopwords)


70
['the', 'you', 'to', 'i', 'a', 'of', 'in', 'your', 'this', 'be', 'is', 'and', 'what', 'it', 'have', 'he', 'for', 'on', 'are', "don't", "i'm", 'that', 'all', "it's", 'not', 'with', 'we', 'my', '(over', 'going', 'get', 'can', 'if', 'but', 'will', 'do', 'come', "you're", 'was', 'out', 'got', 'just', 'see', 'me', "we're", 'about', 'no', 'where', 'you.', 'think', 'there', 'take', 'they', 'at', 'like', 'speaker)', 'has', 'only', "can't", 'know', 'well,', 'been', 'here', 'one', 'red', "i've", 'more', 'luke,', 'as', 'our']


In [25]:
textllll = [w for w in textlll if not w in stopwords]
print(len(stoppwords))


NameError: name 'stoppwords' is not defined

In [ ]:
dist = FreqDist(stoppwords)
items = dist.most_common(30)
for item in items:
  print(item)



In [ ]:
bigramsx = list(nltk.bigrams(textllll))

#print(bigramsx[:20])

bbgram = []
fdist = nltk.FreqDist(bigramsx)
for k,v in fdist.items():
    if float(v) > 4:
        bbgram.append(k)
        bbgram.append(v)
print(bbgram)

dicts = {}
keys = range(4)
values = ["Hi", "I", "am", "John"]
for i in keys:
    for x in values:
        dicts[i] = x
print(dicts)

In [ ]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()



In [ ]:
finder = BigramCollocationFinder.from_words(textlll)
scored = finder.score_ngrams(bigram_measures.raw_freq)


In [ ]:
#print(scored)
type(scored)

scored.sort()

print(sscored)
#sscored =scored.sort(key=lambda tup: tup[2], reverse=True)
#print(sscored)

In [ ]:
def getKey(item):
    return item[1]
#sorted(scored, key=getKey, reverse = True)
sorted(scored, key=getKey, reverse = True)

In [ ]:
finder.apply_word_filter(alpha_filter)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)


In [ ]:
finder.apply_word_filter(lambda w: w in stopwords)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)


In [ ]:
finder2 = BigramCollocationFinder.from_words(textllll)
finder2.apply_freq_filter(5)
scored = finder2.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)




In [ ]:
finder2.apply_ngram_filter(lambda w1, w2: len(w1) > 5)
scored = finder2.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)

In [ ]:
finder3 = BigramCollocationFinder.from_words(textllll)
scored = finder3.score_ngrams(bigram_measures.pmi)
for bscore in scored[:50]:
    print (bscore)

In [ ]:
finder3.apply_freq_filter(5)
scored = finder3.score_ngrams(bigram_measures.pmi)
for bscore in scored[:30]:
    print (bscore)



In [ ]:
import csv
dict = script
w = csv.writer(open("SW5.csv", "w"))
for key, val in dict.items():
    w.writerow([key, val])